In [1]:
pip install openprompt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 146.4/146.4 kB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 6.6 MB/s eta 0:00:00
  Attempting uninstall: sentencepiece
    Found existing installation: sentencepiece 0.1.97
    Uninstalling sentencepiece-0.1.97:
      Successfully uninstalled sentencepiece-0.1.97
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 896.0 kB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [3]:
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
from transformers import TrainingArguments, Trainer
import evaluate
from datasets import load_metric


In [4]:
model_type = "XLM-R" #@param ["SinBERT", "Bert", "XLM-R"]
technique = "hate speech" #@param ["humor", "hate speech", "sentiment"]
random_state = 42 #@param

In [5]:
from openprompt.data_utils.text_classification_dataset import PROCESSORS
from datasets import load_dataset

In [6]:
dataset_path="/kaggle/input/cmcs-dataset-task/ompleted_draft - ompleted_draft.csv"
all_data = pd.read_csv(dataset_path)

if (technique == "humor"):
  all_data = all_data[['Sentence', 'Humor']]
elif (technique == "hate speech"):
  all_data = all_data[['Sentence', 'Hate_speech']]
else:
  all_data = all_data[['Sentence', 'Sentiment']]

all_data.columns = ['Sentence', 'Label']
all_data['Label'], uniq = pd.factorize(all_data['Label'])

X = all_data['Sentence'].values.tolist()
y = all_data['Label'].values.tolist()

In [7]:
# dataset_path="/kaggle/input/sentence-script/sentence - script.csv"
# all_data = pd.read_csv(dataset_path)

# if (technique == "humor"):
#   all_data = all_data[['Sentence', 'Humor',"script"]]
# elif (technique == "hate speech"):
#   all_data = all_data[['Sentence', 'Hate_speech', "script"]]
# else:
#   all_data = all_data[['Sentence', 'Sentiment', "script"]]

# all_data.columns = ['Sentence', 'Label', "script"]
# all_data['Label'], uniq = pd.factorize(all_data['Label'])

# X = all_data[['Sentence',"script"]].values.tolist()
# y = all_data[['Label', "script"]].values.tolist()

In [8]:
# all_data.groupby("script").count()

In [9]:
all_data.groupby("Label").count()


,Sentence
Label,
0,12262
1,348
2,908


In [10]:
uniq

Index(['Not offensive', 'Hate-Inducing', 'Abusive'], dtype='object')

In [11]:
if (technique == 'humor'):
    num_labels=2
    id2label={ 0: "Non-humorous", 1: "Humorous"}
    classes = [ "0","1",]
#     label_words = {"0": ["Non-humorous"],"1": ["Humorous"],}
#     label_words={
#           "0": ["unamusing", "lame","humorless","serious","gloomy","uncomic"],
#           "1": ["funny", "comedic", "amusing", "comic", "comical", "entertaining"],}
    label_words={
        "0": ["නරකයි", "බරපතල","අඳුරු","භයානකයි"],
        "1": ["හාස්‍යය","හාස්‍යජනකයි","විනෝදජනකයි","විනෝදාත්මකයි","විකට"],}
elif (technique == 'hate speech'):
    num_labels=3
    id2label={ 0: "Not offensive", 1: "Hate-Inducing", 2: "Abusive"}
    classes = [ "0","1","2"]
#     label_words={"0": ["ප්‍රසන්න","අහිංසක","හොඳයි"],
#           "1": ["සතුරුකම","ද්වේශ","අකමැති","සතුරු"],
#           "2": ["අපවාදාත්මක","අසභ්‍ය","නින්දා"]}

    label_words={"0": ["pleasant","unoffensive","good"],
          "1": ["animosity","hateful","dislike","hostile"],
          "2": ["abusive","vulgar","derogatory"]}
else:
    num_labels=4
    id2label={ 0: "Negative", 1: "Neutral", 2: "Positive", 3:"Conflict"}
    classes = [ "0","1","2","3"]
#     label_words={"0":["good", "positive","great"],
#                  "1":["bad", "negative","terrible"],
#                  "2":["okay", "neutral"],
#                  "3":["conflict"]}
    label_words={"0":["හොඳයි", "ධනාත්මක"],
                "1":["නරකයි", "ඍණාත්මක"],
                "2":["මධ්යස්ථ"],
                "3":["ගැටුම්කාරී"]}

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state = random_state)

In [13]:
examples=[]
from openprompt.data_utils import InputExample
for i in range(len(X_train)):
  examples.append(InputExample(
        guid = i,
        text_a =X_train[i],
        label=y_train[i]
    ))
# examples=[]
# from openprompt.data_utils import InputExample
# for i in range(len(X_train)):
#   examples.append(InputExample(
#         guid = i,
#         text_a =X_train[i][0],
#         label=y_train[i][0]
#     ))

In [14]:

from transformers.models.auto.tokenization_auto import tokenizer_class_from_name

from openprompt.plms.utils import TokenizerWrapper
from typing import List, Dict
from collections import defaultdict

class MLMTokenizerWrapper(TokenizerWrapper):
    add_input_keys = ['input_ids', 'attention_mask', 'token_type_ids']

    @property
    def mask_token(self):
        return self.tokenizer.mask_token

    @property
    def mask_token_ids(self):
        return self.tokenizer.mask_token_id

    @property
    def num_special_tokens_to_add(self):
        if not hasattr(self, '_num_specials'):
            self._num_specials = self.tokenizer.num_special_tokens_to_add()
        return self._num_specials

    def tokenize_one_example(self, wrapped_example, teacher_forcing):
        ''' # TODO doesn't consider the situation that input has two parts
        '''

        wrapped_example, others = wrapped_example

        # for some dataset like SuperGLUE.COPA, the answer requires prediction an span of
        # the input. Or in generation tasks, we need to generate a piece of target_text.
        # In these case, it tokenized to the encoded_tgt_text for future use.
        encoded_tgt_text = []
        if 'tgt_text' in others:
            tgt_text = others['tgt_text']
            if isinstance(tgt_text, str):
                tgt_text = [tgt_text]
            for t in tgt_text:
                encoded_tgt_text.append(self.tokenizer.encode(t, add_special_tokens=False))


        mask_id = 0 # the i-th the mask token in the template.

        encoder_inputs = defaultdict(list)
        for piece in wrapped_example:
            if piece['loss_ids']==1:
                if teacher_forcing: # fill the mask with the tgt task
                    raise RuntimeError("Masked Language Model can't perform teacher forcing training!")
                else:
                    encode_text = [self.mask_token_ids]
                mask_id += 1

            if piece['text'] in self.special_tokens_maps.keys():
                to_replace = self.special_tokens_maps[piece['text']]
                if to_replace is not None:
                    piece['text'] = to_replace
                else:
                    raise KeyError("This tokenizer doesn't specify {} token.".format(piece['text']))

            if 'soft_token_ids' in piece and piece['soft_token_ids']!=0:
                encode_text = [0] # can be replace by any token, since these token will use their own embeddings
            else:
                encode_text = self.tokenizer.encode(piece['text'], add_special_tokens=False)

            encoding_length = len(encode_text)
            encoder_inputs['input_ids'].append(encode_text)
            for key in piece:
                if key not in ['text']:
                    encoder_inputs[key].append([piece[key]]*encoding_length)

        encoder_inputs = self.truncate(encoder_inputs=encoder_inputs)
        # delete shortenable ids
        encoder_inputs.pop("shortenable_ids")
        encoder_inputs = self.concate_parts(input_dict=encoder_inputs)
        encoder_inputs = self.add_special_tokens(encoder_inputs=encoder_inputs)
        # create special input ids
        encoder_inputs['attention_mask'] = [1] *len(encoder_inputs['input_ids'])
        if self.create_token_type_ids:
            encoder_inputs['token_type_ids'] = [0] *len(encoder_inputs['input_ids'])
        # padding
        encoder_inputs = self.padding(input_dict=encoder_inputs, max_len=self.max_seq_length, pad_id_for_inputs=self.tokenizer.pad_token_id)


        if len(encoded_tgt_text) > 0:
            encoder_inputs = {**encoder_inputs, "encoded_tgt_text": encoded_tgt_text}# convert defaultdict to dict
        else:
            encoder_inputs = {**encoder_inputs}
        return encoder_inputs

In [15]:
from statistics import mode
from typing import List, Optional
from transformers.modeling_utils import PreTrainedModel
from transformers.tokenization_utils import PreTrainedTokenizer
from transformers import BertConfig, BertTokenizer, BertModel, BertForMaskedLM, \
                         RobertaConfig, RobertaTokenizer, RobertaModel, RobertaForMaskedLM, \
                         XLMRobertaConfig, XLMRobertaTokenizer, XLMRobertaModel, XLMRobertaForMaskedLM
from collections import namedtuple
from yacs.config import CfgNode

from openprompt.utils.logging import logger


ModelClass = namedtuple("ModelClass", ('config', 'tokenizer', 'model','wrapper'))

_MODEL_CLASSES = {
    'bert': ModelClass(**{
        'config': BertConfig,
        'tokenizer': BertTokenizer,
        'model':BertForMaskedLM,
        'wrapper': MLMTokenizerWrapper,
    }),
    'roberta': ModelClass(**{
        'config': RobertaConfig,
        'tokenizer': RobertaTokenizer,
        'model':RobertaForMaskedLM,
        'wrapper': MLMTokenizerWrapper
    }),
    'xlm': ModelClass(**{
        'config': XLMRobertaConfig,
        'tokenizer': XLMRobertaTokenizer,
        'model': XLMRobertaForMaskedLM,
        'wrapper': MLMTokenizerWrapper
    }),
}


def get_model_class(plm_type: str):
    return _MODEL_CLASSES[plm_type]


def load_plm(model_name, model_path, specials_to_add = None):
    r"""A plm loader using a global config.
    It will load the model, tokenizer, and config simulatenously.

    Args:
        config (:obj:`CfgNode`): The global config from the CfgNode.

    Returns:
        :obj:`PreTrainedModel`: The pretrained model.
        :obj:`tokenizer`: The pretrained tokenizer.
        :obj:`model_config`: The config of the pretrained model.
        :obj:`wrapper`: The wrapper class of this plm.
    """
    model_class = get_model_class(plm_type = model_name)
    model_config = model_class.config.from_pretrained(model_path)
    # you can change huggingface model_config here
    # if 't5'  in model_name: # remove dropout according to PPT~\ref{}
    #     model_config.dropout_rate = 0.0
    if 'gpt' in model_name: # add pad token for gpt
        specials_to_add = ["<pad>"]
        # model_config.attn_pdrop = 0.0
        # model_config.resid_pdrop = 0.0
        # model_config.embd_pdrop = 0.0
    model = model_class.model.from_pretrained(model_path, config=model_config)
    tokenizer = model_class.tokenizer.from_pretrained(model_path)
    wrapper = model_class.wrapper


    model, tokenizer = add_special_tokens(model, tokenizer, specials_to_add=specials_to_add)

    if 'opt' in model_name:
        tokenizer.add_bos_token=False
    return model, tokenizer, model_config, wrapper




def load_plm_from_config(config: CfgNode):
    r"""A plm loader using a global config.
    It will load the model, tokenizer, and config simulatenously.

    Args:
        config (:obj:`CfgNode`): The global config from the CfgNode.

    Returns:
        :obj:`PreTrainedModel`: The pretrained model.
        :obj:`tokenizer`: The pretrained tokenizer.
        :obj:`model_config`: The config of the pretrained model.
        :obj:`model_config`: The wrapper class of this plm.
    """
    plm_config = config.plm
    model_class = get_model_class(plm_type = plm_config.model_name)
    model_config = model_class.config.from_pretrained(plm_config.model_path)
    # you can change huggingface model_config here
    # if 't5'  in plm_config.model_name: # remove dropout according to PPT~\ref{}
    #     model_config.dropout_rate = 0.0
    if 'gpt' in plm_config.model_name: # add pad token for gpt
        if "<pad>" not in config.plm.specials_to_add:
            config.plm.specials_to_add.append("<pad>")
    model = model_class.model.from_pretrained(plm_config.model_path, config=model_config)
    tokenizer = model_class.tokenizer.from_pretrained(plm_config.model_path)
    wrapper = model_class.wrapper
    model, tokenizer = add_special_tokens(model, tokenizer, specials_to_add=config.plm.specials_to_add)
    return model, tokenizer, model_config, wrapper

def add_special_tokens(model: PreTrainedModel,
                       tokenizer: PreTrainedTokenizer,
                       specials_to_add: Optional[List[str]] = None):
    r"""add the special_tokens to tokenizer if the special token
    is not in the tokenizer.

    Args:
        model (:obj:`PreTrainedModel`): The pretrained model to resize embedding
                after adding special tokens.
        tokenizer (:obj:`PreTrainedTokenizer`): The pretrained tokenizer to add special tokens.
        specials_to_add: (:obj:`List[str]`, optional): The special tokens to be added. Defaults to pad token.

    Returns:
        The resized model, The tokenizer with the added special tokens.

    """
    if specials_to_add is None:
        return model, tokenizer
    for token in specials_to_add:
        if "pad" in token.lower():
            if tokenizer.pad_token is None:
                tokenizer.add_special_tokens({'pad_token': token})
                model.resize_token_embeddings(len(tokenizer))
                logger.info("pad token is None, set to id {}".format(tokenizer.pad_token_id))
    return model, tokenizer

In [16]:
plm, tokenizer, model_config, WrapperClass =load_plm("xlm", "xlm-roberta-base")

Downloading:   0%|          | 0.00/615 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.04G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.83M [00:00<?, ?B/s]

In [17]:
from openprompt.prompts import MixedTemplate
promptTemplate = MixedTemplate(
    model=plm,
    text = '{"placeholder": "text_a"} {"soft": "It"} {"soft": "was"} {"mask"}.',
    tokenizer = tokenizer,
)

# from openprompt.prompts import ManualTemplate
# promptTemplate = ManualTemplate(
#     text = '{"placeholder": "text_a"}. ඒක {"mask"}.',
#     tokenizer = tokenizer,
    
# )

In [18]:
from openprompt.prompts import SoftVerbalizer
promptVerbalizer = SoftVerbalizer(tokenizer, plm, num_classes=num_labels)

# from openprompt.prompts import ManualVerbalizer

# promptVerbalizer = ManualVerbalizer(
#     classes = classes,
#     label_words = label_words,
#     tokenizer = tokenizer,
# )

# Non-humorous', 'Humorous

In [19]:
from openprompt import PromptForClassification
promptModel = PromptForClassification(
    template = promptTemplate,
    plm = plm,
    verbalizer = promptVerbalizer,
)

In [20]:
MAX_LEN = 128

from openprompt import PromptDataLoader
data_loader = PromptDataLoader(
    dataset = examples,
    tokenizer = tokenizer,
    template = promptTemplate,
    tokenizer_wrapper_class=WrapperClass,
    batch_size=5,
    max_length=MAX_LEN,
    truncation=True,
    padding="max_length"
)

tokenizing: 12166it [00:07, 1551.91it/s]


In [21]:
examples_test=[]
from openprompt.data_utils import InputExample
for i in range(len(X_test)):
  examples_test.append(InputExample(
        guid = i,
        text_a =X_test[i],
        label=y_test[i]
    ))

# latin_examples_test=[]
# sinhala_examples_test=[]
# mixed_examples_test=[]
# from openprompt.data_utils import InputExample
# for i in range(len(X_test)):
#   if(X_test[i][1]=="Latin"):
#       latin_examples_test.append(InputExample(
#             guid = i,
#             text_a =X_test[i][0],
#             label=y_test[i][0]
#         ))
#   if(X_test[i][1]=="Sinhala"):
#       sinhala_examples_test.append(InputExample(
#             guid = i,
#             text_a =X_test[i][0],
#             label=y_test[i][0]
#         ))   
        
#   if(X_test[i][1]=="Mixed"):
#       mixed_examples_test.append(InputExample(
#             guid = i,
#             text_a =X_test[i][0],
#             label=y_test[i][0]
#         ))
    
  

In [22]:
import numpy as np
import torch

class EarlyStopping:
    """Early stops the training if validation loss doesn't improve after a given patience."""
    def __init__(self, patience=7, verbose=False, delta=0, path='checkpoint.pt', trace_func=print):
        """
        Args:
            patience (int): How long to wait after last time validation loss improved.
                            Default: 7
            verbose (bool): If True, prints a message for each validation loss improvement. 
                            Default: False
            delta (float): Minimum change in the monitored quantity to qualify as an improvement.
                            Default: 0
            path (str): Path for the checkpoint to be saved to.
                            Default: 'checkpoint.pt'
            trace_func (function): trace print function.
                            Default: print            
        """
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = np.Inf
        self.delta = delta
        self.path = path
        self.trace_func = trace_func
    def __call__(self, val_loss, model):

        score = -val_loss

        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
        elif score < self.best_score + self.delta:
            self.counter += 1
            self.trace_func(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
            self.counter = 0

    def save_checkpoint(self, val_loss, model):
        '''Saves model when validation loss decrease.'''
        if self.verbose:
            self.trace_func(f'Validation loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f}).  Saving model ...')
        torch.save(model.state_dict(), self.path)
        self.val_loss_min = val_loss

In [23]:
data_loader_test = PromptDataLoader(
    dataset = examples_test,
    tokenizer = tokenizer,
    template = promptTemplate,
    tokenizer_wrapper_class=WrapperClass,
    batch_size=1,
    max_length=MAX_LEN,
    truncation=True,
    padding="max_length"
)

tokenizing: 1352it [00:00, 1538.05it/s]


In [24]:
import torch
from transformers import  AdamW, get_linear_schedule_with_warmup
loss_func = torch.nn.CrossEntropyLoss()
optimizer_grouped_parameters = [
    {'params': [p for n,p in promptModel.template.named_parameters() if "raw_embedding" not in n]}
]
optimizer = AdamW(optimizer_grouped_parameters, lr=1e-3)

optimizer_grouped_parameters1 = [
    {'params': promptModel.verbalizer.group_parameters_1, "lr":3e-5},
    {'params': promptModel.verbalizer.group_parameters_2, "lr":3e-4},
]

optimizer1 = AdamW(optimizer_grouped_parameters1)


/opt/conda/lib/python3.7/site-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


In [25]:
n_epochs=5
# to track the training loss as the model trains
train_losses = []
# to track the validation loss as the model trains
valid_losses = []
# to track the average training loss per epoch as the model trains
avg_train_losses = []
# to track the average validation loss per epoch as the model trains
avg_valid_losses = [] 

# initialize the early_stopping object
early_stopping = EarlyStopping(patience=2, verbose=True)

promptModel=promptModel.cuda()
for epoch in range(n_epochs):
    tot_loss = 0
    for step, inputs in enumerate(data_loader):
        inputs = inputs.cuda()
        logits = promptModel(inputs)
        labels = inputs['label']
        loss = loss_func(logits, labels)
        loss.backward()
        tot_loss += loss.item()
        optimizer.step()
        optimizer.zero_grad()
        optimizer1.step()
        optimizer1.zero_grad()
        train_losses.append(loss.item())

    for step, inputs in enumerate(data_loader_test):
        inputs=inputs.cuda();
        logits = promptModel(inputs)    
        labels = inputs['label']
        loss = loss_func(logits, labels)
        valid_losses.append(loss.item())

    train_loss = np.average(train_losses)
    valid_loss = np.average(valid_losses)
    avg_train_losses.append(train_loss)
    avg_valid_losses.append(valid_loss)
        
    epoch_len = len(str(n_epochs))
        
    print_msg = (f'[{epoch:>{epoch_len}}/{n_epochs:>{epoch_len}}] ' +
                     f'train_loss: {train_loss:.5f} ' +
                     f'valid_loss: {valid_loss:.5f}')
        
    print(print_msg)
        
        # clear lists to track next epoch
    train_losses = []
    valid_losses = []
        
        # early_stopping needs the validation loss to check if it has decresed, 
        # and if it has, it will make a checkpoint of the current model
    early_stopping(valid_loss, promptModel)
        
    if early_stopping.early_stop:
        print("Early stopping")
        break
        
# load the last checkpoint with the best model
promptModel.load_state_dict(torch.load('checkpoint.pt'))

[0/5] train_loss: 0.31318 valid_loss: 0.35013
Validation loss decreased (inf --> 0.350128).  Saving model ...
[1/5] train_loss: 0.28070 valid_loss: 0.32865
Validation loss decreased (0.350128 --> 0.328651).  Saving model ...
[2/5] train_loss: 0.26969 valid_loss: 0.31509
Validation loss decreased (0.328651 --> 0.315094).  Saving model ...
[3/5] train_loss: 0.25722 valid_loss: 0.30615
Validation loss decreased (0.315094 --> 0.306150).  Saving model ...
[4/5] train_loss: 0.24826 valid_loss: 0.30100
Validation loss decreased (0.306150 --> 0.301000).  Saving model ...


<All keys matched successfully>

In [26]:
# promptModel=promptModel.cuda()
# for epoch in range(5):
#     tot_loss = 0
#     for step, inputs in enumerate(data_loader):
#         inputs = inputs.cuda()
#         logits = promptModel(inputs)
#         labels = inputs['label']
#         loss = loss_func(logits, labels)
#         loss.backward()
#         tot_loss += loss.item()
#         optimizer.step()
#         optimizer.zero_grad()
#         optimizer1.step()
#         optimizer1.zero_grad()
#         print(tot_loss/(step+1))

In [27]:
# latin_data_loader = PromptDataLoader(
#     dataset = latin_examples_test,
#     tokenizer = tokenizer,
#     template = promptTemplate,
#     tokenizer_wrapper_class=WrapperClass,
#     batch_size=1,
#     max_length=MAX_LEN,
#     truncation=True,
#     padding="max_length"
# )

In [28]:
# sinhala_data_loader = PromptDataLoader(
#     dataset = sinhala_examples_test,
#     tokenizer = tokenizer,
#     template = promptTemplate,
#     tokenizer_wrapper_class=WrapperClass,
#     batch_size=1,
#     max_length=MAX_LEN,
#     truncation=True,
#     padding="max_length"
# )

In [29]:
# mixed_data_loader = PromptDataLoader(
#     dataset = mixed_examples_test,
#     tokenizer = tokenizer,
#     template = promptTemplate,
#     tokenizer_wrapper_class=WrapperClass,
#     batch_size=1,
#     max_length=MAX_LEN,
#     truncation=True,
#     padding="max_length"
# )

In [30]:
# promptModel=promptModel.cuda()

def test_func(data_loader):
    import torch
    torch.cuda.empty_cache()
    allpreds = []
    alllabels = []
    for step, inputs in enumerate(data_loader):
        inputs=inputs.cuda();
        logits = promptModel(inputs)    
        labels = inputs['label']
        alllabels.extend(labels.cpu().tolist())
        allpreds.extend(torch.argmax(logits, dim=-1).cpu().tolist())
    return allpreds,alllabels

In [31]:

def compute_metrics(allpreds,alllabels):
    metric1 = load_metric("precision")
    metric2 = load_metric("recall")
    metric3 = load_metric("f1")
    metric4 = load_metric("accuracy")
    
    predictions, labels = allpreds,alllabels
    precision = metric1.compute(predictions=predictions, references=labels, average="weighted")["precision"]
    recall = metric2.compute(predictions=predictions, references=labels, average="weighted")["recall"]
    f1 = metric3.compute(predictions=predictions, references=labels, average="weighted")["f1"]
    accuracy = metric4.compute(predictions=predictions, references=labels)["accuracy"]
    macro_precision = metric1.compute(predictions=predictions, references=labels, average="macro")["precision"]
    macro_recall = metric2.compute(predictions=predictions, references=labels, average="macro")["recall"]
    macro_f1 = metric3.compute(predictions=predictions, references=labels, average="macro")["f1"]
    return {"accuracy":accuracy, "precision": precision, "recall": recall, "f1": f1, "macro_precision": macro_precision, "macro_recall": macro_recall, "macro_f1": macro_f1}

In [32]:
allpreds,alllabels=test_func(data_loader)
compute_metrics(allpreds,alllabels)

{'accuracy': 0.9021864211737629,
 'precision': 0.9034639543064547,
 'recall': 0.9021864211737629,
 'f1': 0.9003275417791224,
 'macro_precision': 0.6278448275298887,
 'macro_recall': 0.5477822106288265,
 'macro_f1': 0.5583755285016724}

In [33]:
# allpreds,alllabels=test_func(latin_data_loader)
# compute_metrics(allpreds,alllabels)

In [34]:
# allpreds,alllabels=test_func(sinhala_data_loader)
# compute_metrics(allpreds,alllabels)

In [35]:
# allpreds,alllabels=test_func(mixed_data_loader)
# compute_metrics(allpreds,alllabels)

In [36]:
# c=0
# for i,j in zip(allpreds, alllabels):
#   if(i!=j):
#     print(c,X_test[c],"---->","predicted :", uniq[i]," actual :",  uniq[j])

#   c+=1

In [37]:
# c=0
# for i,j in zip(allpreds, alllabels):
#   if(i==j):
#     print(c,X_test[c],"---->","predicted :", uniq[i]," actual :",  uniq[j])
    
#   c+=1